## TFBertforSequenceClassification
이전보다 훨씬 간소화 된 코드로 네이버 영화 리뷰 분류 실습을 진행해봅시다.

In [1]:
!pip install tf_keras

y## 1. 데이터 로드 및 정제
이부분은 바로 이저'BERT를 이용한 네이버 영화 리뷰 분류 실습과 동일

In [2]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tf_keras as keras
from transformers import BertTokenizer, TFBertForSequenceClassification


In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7a9080df6e10>)

In [4]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

print('훈련용 리뷰 개수: ', len(train_data))
print('테스트용 리뷰 개수: ', len(test_data))

훈련용 리뷰 개수:  150000
테스트용 리뷰 개수:  50000


훈련 데이터와 테스트 데이터의 리뷰 개수는 각각 15만개와 5만개입니다. 중복 데이터와 결측값을 제거 합니다.

In [5]:
# 훈련 데이터 중복값, 결측값 제거
train_data.drop_duplicates(subset=['document'], inplace=True)
train_data = train_data.dropna(how='any')
print('훈련용 리뷰 개수: ', len(train_data))

# 테스트 데이터 결측값 제거
test_data = test_data.dropna(how = 'any')
print('테 스 트 데 이 터 의 리 뷰 수 :',len(test_data))

훈련용 리뷰 개수:  146182
테 스 트 데 이 터 의 리 뷰 수 : 49997


### 토크나이저를 이용한 정수 인코딩
transformers 에서 제공하는 BertTokenizerFast 를 사용하여 KLUE‐BERT 의 토크나이저를 로드합니다.

In [6]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("klue/bert-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

훈련 데이터와 테스트 데이터로부터 리뷰 데이터와 레이블을 별도로 분리하여 저장합니다.

In [7]:
X_train_list = train_data['document'].tolist()
y_train = train_data['label'].tolist()
X_test_list = test_data['document'].tolist()
y_test = test_data['label'].tolist()

BertTokenizerFast 토크나이저를 사용하여 정수 인코딩과 패딩을 수행합니다. 이전 실습에서 세그먼트
인코딩과 어텐션 마스크를 직접 만들어준 것과는 달리 이번에는 토크나이저를 통해 자동으로 만들어보
겠습니다.

In [8]:
X_train = tokenizer(X_train_list, truncation=True, padding=True)
X_test = tokenizer(X_test_list, truncation=True, padding=True)

이제 X_train 과 X_test 에는 정수 인코딩과 패딩된 결과가 저장되어져 있습니다. 훈련 데이터의 첫번째 리
뷰인 X_train[0] 을 예시로 여러가지 값들을 확인할 수 있는 방법을 알아봅시다. 우선 샘플에.tokens 를 사
용하면 토큰화 결과를 확인가능합니다.

In [9]:
print(X_train[0].tokens)

['[CLS]', '아', '더', '##빙', '.', '.', '진짜', '짜증', '##나', '##네', '##요', '목소리', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD

샘플에 .ids를 하면 정수 인코딩 결과를 확인 할수 있습니다.

In [10]:
print(X_train[0].ids)

[2, 1376, 831, 2604, 18, 18, 4229, 9801, 2075, 2203, 2182, 4243, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [11]:
print(X_train[0].type_ids)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


이번에는 실제 단어가 있는 위치와 패딩의 위치를 BERT 에게 알려주는 역할을 하는 어텐션 마스크입니
다. 이는 각 샘플에.attention_mask 를 사용하여 확인할 수 있습니다.

In [12]:
print(X_train[0].attention_mask)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


앞서 정수 인코딩 결과를 보면 2, 1376, 831, 2604, 18, 18, 4229, 9801, 2075, 2203, 2182, 4243, 3 라는 총
13 개의 정수 다음에는 패딩을 위한 0 의 값이 나열되고 있습니다. 어텐션 마스크에서도 앞의 13 개의 위
치에는 1 이 들어가고, 나머지는 전부 0 의 값이 나열됩니다.
BERT 의 입력으로 각 샘플마다 정수 인코딩 결과인 ids, 문장의 종류가 몇 개인지를 알려주는 type_ids,
실제 단어와 패딩의 위치를 구분할 수 있도록 하는 attention_mask 이 3 개가 사용됩니다. 첫번째 샘플인
X_train[0] 이 아니라 다른 샘플에 대해서도 직접 출력하여 확인해보시기 바랍니다.

In [13]:
print(X_train[5].ids)
print(X_train[5].type_ids)
print(X_train[5].attention_mask)

[2, 1037, 29199, 1, 23, 2103, 3797, 4885, 4622, 21, 18458, 2065, 2179, 28, 2593, 2048, 16516, 18, 3901, 18, 18, 18, 21320, 2019, 2119, 5663, 2894, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

### 3. 데이터셋 생성 및 모델 학습

In [14]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification
from tf_keras.callbacks import EarlyStopping

데이터를 텐서플로우의 데이터셋 형태로 변환합니다.

In [15]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(X_train), y_train))
valid_dataset = tf.data.Dataset.from_tensor_slices((dict(X_test), y_test))

옵티마이저는 Adam을 사용합니다.

In [16]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

transformers 에서는 텍스트 분류를 위한 모델 구현체인 TFBertForSequenceClassification 를 제공합니
다.

- TFBertForSequenceClassification.from_pretrained(‘모델 이름’, num_labels= 분류할 레이블의
수)

만약 위 코드를 사용하지 않을 경우에는 앞에서 진행한 ‘한국어 BERT 를 이용한 네이버 영화 리뷰 분류’ 실
습과 같이 BERT 의 마지막 층의 ‘CLS 토큰’ 위치의 벡터를 꺼내와서 출력층으로 연결하는 일을 사용자가
직접 해주어야 합니다.

In [17]:
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base",num_labels=2, from_pt=True)
model.compile(optimizer='Adam', loss=model.hf_compute_loss, metrics=['accuracy'])

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should p

loss 의 인자값으로는 model.hf_compute_loss 를 사용하고 있는데, TFBertForSequenceClassification
의 hf_compute_loss 는 우리가 다중 클래스 분류에서 사용하던 손실 함수인 크로스 엔트로피 함수가 맵
핑되어져 있습니다.
다수의 에포크 학습을 사용한다면 콜백으로 조기 종료 (Early Stopping) 를 사용해볼 수 있겠습니다. 여기
서는 min_delta 를 사용했는데, 예를 들어, min_delta 가 0.01 이고, 2 에포크 정확도가 0.8122 라고 할 때,
만약 3 에포크에 정확도가 0.8129 라고 하면 이는 0.007 의 개선이 있었지만 min_delta 의 값 0.01 에는 미치지 못했으므로 개선된 것으로 보지 않습니다. 저자의 경우에는 2 에포크만 수행하지만 독자분들의 경
우에는 에포크를 늘려보시기 바랍니다.

In [18]:
import tensorflow as tf
from tf_keras.callbacks import EarlyStopping

print(tf.__version__)       # TensorFlow 버전 확인
print(tf.keras.__version__) # Keras 버전 확인

2.18.0
3.8.0


In [19]:
early_stopping = EarlyStopping(
monitor="val_accuracy",
min_delta=0.001,
patience=2)

model.fit(train_dataset.shuffle(10000).batch(32), epochs=2, batch_size=32,
          validation_data = valid_dataset.shuffle(10000). batch(32),
          callbacks = [early_stopping])

Epoch 1/2
4569/4569 [==============================] - 2071s 427ms/step - loss: 0.7112 - accuracy: 0.5016 - val_loss: 0.6978 - val_accuracy: 0.4965
Epoch 2/2
4569/4569 [==============================] - 1910s 418ms/step - loss: 0.6963 - accuracy: 0.5019 - val_loss: 0.6931 - val_accuracy: 0.5035


In [20]:
model.evaluate(valid_dataset.batch(1024))

49/49 [==============================] - 95s 2s/step - loss: 0.6931 - accuracy: 0.5035


[0.6931354403495789, 0.5034502148628235]

In [21]:
model.save_pretrained('nsmc_model/bert-base')
tokenizer.save_pretrained('nsmc_model/bert-base')

('nsmc_model/bert-base/tokenizer_config.json',
 'nsmc_model/bert-base/special_tokens_map.json',
 'nsmc_model/bert-base/vocab.txt',
 'nsmc_model/bert-base/added_tokens.json',
 'nsmc_model/bert-base/tokenizer.json')

In [22]:
from transformers import TextClassificationPipeline

In [23]:
loaded_tokenizer = BertTokenizerFast.from_pretrained('nsmc_model/bert-base')

loaded_model = TFBertForSequenceClassification.from_pretrained('nsmc_model/bert-base')

Some layers from the model checkpoint at nsmc_model/bert-base were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at nsmc_model/bert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [24]:
text_classifier = TextClassificationPipeline(
tokenizer=loaded_tokenizer,
model=loaded_model,
framework='tf',
return_all_scores=True
)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Device set to use 0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [25]:
text_classifier('뭐 야 이 평 점 들 은.... 나 쁘 진 않 지 만 10점 짜 리 는 더 더 욱 아 니 잖 아')
[0]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


[0]